In [1]:
import pandas as pd
import yfinance as yf
import numpy as np

## Inputs
Stock = input("Please input the stock you want to run the discounted cashflow model on (example: aapl):")
Forecasting_Time = int(input("Please input the amount of time you want forecasted:"))
perpetuity_percentage = float(input("Please input the perpetuity percentage:(example: 0.02)"))


## First we want to get the stock information for our specific security.
Ticker = yf.Ticker(Stock)
cashflow_df = Ticker.cashflow

# This takes the cashflow based off the previous 5 years (Yfinance only gets the past 5 years worth)
# yfinance also gives it to us in a decreasing order, so we need to rotate it.

YTD_Cashflow = (cashflow_df.iloc[0].dropna())[::-1]

## We also need the number of stocks that are currently oustanding.

Stock_Info = Ticker.info
SharesOutstanding = Stock_Info["sharesOutstanding"]

## We now want to calculate the percentage increase or decrease in the company's cashflow as an average over time
## I will find a better way to forecast cashflow growth.
cash_change_average = ((YTD_Cashflow.pct_change()).mean() * 100) / 100
bearish_cash_average = cash_change_average - 0.02
bullish_cash_average = cash_change_average + 0.02


## We need to calculate Cost of Equity
## (Cost of equity = RisklessRate + Beta*Equity Risk Premium)
## We find the Beta, Risk Free Rate and Risk Premium


Ten_Year_Treasury = yf.Ticker('^tnx')
Ten_Year_Treasury_Rate = Ten_Year_Treasury.history(period = 'max')
Most_Recent_Rate = Ten_Year_Treasury_Rate["Close"].iloc[-1] / 100
Equity_Risk_Premium = 0.00535
Beta = Stock_Info["beta"]


Cost_of_Equity = Most_Recent_Rate + Beta * Equity_Risk_Premium


## We also want The Cost of Capital
## Cost of Capital = Risk_Free_Rate * (1 - Tax Rate)
## We will use the companies income statements for the tax rate.


Income_Statement = Ticker.financials
Tax_Rates = Income_Statement.iloc[1][0]


Cost_of_Capital = Most_Recent_Rate * (1-Tax_Rates)

## Finally we calculate the Weighted Average Cost of Capital
## We need the debt and equity weight for the specific company, which we will get through yfinance again.
## WACC = debt_weight * Cost of Capital + equity_weight * Cost of Equity
## Debt_weight = Total Debt / (Total Debt + Total Equity)


Balance_Sheet = Ticker.balancesheet
Total_Debt = Balance_Sheet.iloc[4][0]
Total_Equity = Stock_Info["marketCap"]
Debt_Weight = Total_Debt / (Total_Debt + Total_Equity)
Equity_Weight = 1 - Debt_Weight

WACC = Debt_Weight * Cost_of_Capital + Equity_Weight * Cost_of_Equity

# ## After this we need to forecast ahead by creating a new dataframe and adding dates.
def future_cashflow(bearish_cash_change, normal_cash_change, bullish_cash_change, forecast_period):
   bearish_future_values = [YTD_Cashflow[-1]]
   normal_future_values = [YTD_Cashflow[-1]]
   bullish_future_values = [YTD_Cashflow[-1]]
   for cash in range(forecast_period):
       cashflows =  bearish_future_values[-1] * (1 + bearish_cash_change)
       bearish_future_values.append(cashflows)
   for cash in range(forecast_period):
       cashflows = normal_future_values[-1] * (1 + normal_cash_change)
       normal_future_values.append(cashflows)
   for cash in range(forecast_period):
       cashflows = bullish_future_values[-1] * (1 + bullish_cash_change)
       bullish_future_values.append(cashflows)


   return [bearish_future_values[1:] , normal_future_values[1:] , bullish_future_values[1:]]


## Here we calculate the discounted and perpetuity cashflow amounts
def discounted_cashflow(perpetuity_percentage, future_cashflow, required_rate):
   list_of_cashflows = future_cashflow(bearish_cash_average,cash_change_average,bullish_cash_average,Forecasting_Time)
   bearish_discounted_values = []
   normal_discounted_values = []
   bullish_discounted_values = []
   terminal_value = YTD_Cashflow[-1] * ((1 + perpetuity_percentage)/(required_rate - perpetuity_percentage))


   for t, cashflow in enumerate(list_of_cashflows[0]):
       discounted_amount = cashflow / ((1 + required_rate) ** (t + 1))
       bearish_discounted_values.append(discounted_amount)
   for t, cashflow in enumerate(list_of_cashflows[1]):
       discounted_amount = cashflow / ((1 + required_rate) ** (t + 1))
       normal_discounted_values.append(discounted_amount)
   for t, cashflow in enumerate(list_of_cashflows[2]):
       discounted_amount = cashflow / ((1 + required_rate) ** (t + 1))
       bullish_discounted_values.append(discounted_amount)
  
   bearish_discounted_values.append(terminal_value / (1 + required_rate) ** len(list_of_cashflows[0]) + 1)
   normal_discounted_values.append(terminal_value / (1 + required_rate) ** len(list_of_cashflows[1]) + 1)
   bullish_discounted_values.append(terminal_value / (1 + required_rate) ** len(list_of_cashflows[2]) + 1)


   return [sum(bearish_discounted_values) , sum(normal_discounted_values) , sum(bullish_discounted_values)]


## this simply just divides by the outstanding shares numbers.
  
def Instrinsic_Value(discounted_cashflow,number_of_stocks):
   Values = []
   list_of_discounts = discounted_cashflow(perpetuity_percentage,future_cashflow,WACC)
   for discount_cash in list_of_discounts:
       intrinsic_price = discount_cash / number_of_stocks
       Values.append(intrinsic_price)
   Prices = pd.Series(Values)
   Prices.index = ["Bearish","Normal","Bullish"]
   return Prices


Instrinsic_Value(discounted_cashflow,SharesOutstanding)



C:\Users\Family\AppData\Local\Temp\ipykernel_24380\630720090.py:41: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cash_change_average = ((YTD_Cashflow.pct_change()).mean() * 100) / 100
C:\Users\Family\AppData\Local\Temp\ipykernel_24380\630720090.py:71: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Tax_Rates = Income_Statement.iloc[1][0]


0.04388071554200068


C:\Users\Family\AppData\Local\Temp\ipykernel_24380\630720090.py:86: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Total_Debt = Balance_Sheet.iloc[4][0]
C:\Users\Family\AppData\Local\Temp\ipykernel_24380\630720090.py:98: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  bearish_future_values = [YTD_Cashflow[-1]]
C:\Users\Family\AppData\Local\Temp\ipykernel_24380\630720090.py:99: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  normal_future

Bearish    264.104713
Normal     266.327695
Bullish    268.663540
dtype: float64